<a href="https://colab.research.google.com/github/hariprasad7/hari_INFO5731_Spring2021/blob/main/In_class_exercise_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The fifth In-class-exercise (2/23/2021, 20 points in total)

In exercise-03, I asked you to collected 500 textual data based on your own information needs (If you didn't collect the textual data, you should recollect for this exercise). Now we need to think about how to represent the textual data for text classification. In this exercise, you are required to select 10 types of features (10 types of features but absolutely more than 10 features) in the followings feature list, then represent the 500 texts with these features. The output should be in the following format:
![image.png](attachment:image.png)

The feature list:

* (1) tf-idf features
* (2) POS-tag features: number of adjective, adverb, auxiliary, punctuation, complementizer, coordinating conjunction, subordinating conjunction, determiner, interjection, noun, possessor, preposition, pronoun, quantifier, verb, and other. (select some of them if you use pos-tag features)
* (3) Linguistic features:
  * number of right-branching nodes across all constituent types
  * number of right-branching nodes for NPs only
  * number of left-branching nodes across all constituent types
  * number of left-branching nodes for NPs only
  * number of premodifiers across all constituent types
  * number of premodifiers within NPs only
  * number of postmodifiers across all constituent types
  * number of postmodifiers within NPs only
  * branching index across all constituent types, i.e. the number of right-branching nodes minus number of left-branching nodes
  * branching index for NPs only
  * branching weight index: number of tokens covered by right-branching nodes minus number of tokens covered by left-branching nodes across all categories
  * branching weight index for NPs only 
  * modification index, i.e. the number of premodifiers minus the number of postmodifiers across all categories
  * modification index for NPs only
  * modification weight index: length in tokens of all premodifiers minus length in tokens of all postmodifiers across all categories
  * modification weight index for NPs only
  * coordination balance, i.e. the maximal length difference in coordinated constituents
  
  * density (density can be calculated using the ratio of folowing function words to content words) of determiners/quantifiers
  * density of pronouns
  * density of prepositions
  * density of punctuation marks, specifically commas and semicolons
  * density of auxiliary verbs
  * density of conjunctions
  * density of different pronoun types: Wh, 1st, 2nd, and 3rd person pronouns
  
  * maximal and average NP length
  * maximal and average AJP length
  * maximal and average PP length
  * maximal and average AVP length
  * sentence length

* Other features in your mind (ie., pre-defined patterns)

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
def forurl(page_num):
  if page_num>=10:
    url=f'https://citeseerx.ist.psu.edu/search?q=natural+language+processing&t=doc&sort=rlv&start={page_num}'
  else:
    url="https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc"
  req = requests.get(url)
  soup = BeautifulSoup(req.content, 'html.parser')
  soup1 = soup.find_all('div', {'class': 'pubabstract'})
  list1=[]
  for abstract in soup1:
    list1.append(abstract.text)
  return list1
list2=[]
for i in range(0,500,10):
  list2=list2+forurl(i)
df=pd.DataFrame(list2)
print(df)
df.to_csv('abstract.csv')

                                                     0
0    \n                    Abstract not found\n    ...
1    \n                     describe a method for s...
2    \n                    Scaling conditional rand...
3    \n                    The paper addresses the ...
4    \n                    In most natural language...
..                                                 ...
495  \n                    ABSTRACT: Web service us...
496  \n                    Much of the pre-existing...
497  \n                     from sources with a hie...
498  \n                    . But learners get annoy...
499  \n                     personalities and facia...

[500 rows x 1 columns]


In [10]:
df.columns=["Abstracts"]
#Removing special characters and punctuations
df['special characters and punctuations removal']=df['Abstracts'].str.replace('[!-;:,<>./?@#$%^&*_~{}]','')
#Remove numbers
df['Numbers removal']=df['Abstracts'].str.replace('\d','')
#Remove stopwords
f=open('Stopwords.txt','r')#pulled stopwords provided
a=f.readlines()
list3=[]
for i in a:
  list3.append(i.strip())
df['Stopwords removal']=df['Abstracts'].apply(lambda x: " ".join(x for x in x.split() if x not in list3))
#Lower casing
df['Lower casing']=df['Abstracts'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#stemming
from nltk.stem import PorterStemmer
st=PorterStemmer()
df['Stemming']=df['Abstracts'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
#Lemmatization
import nltk
from textblob import Word
nltk.download('wordnet')
df['Lemmatization']=df['Abstracts'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df.to_csv('data_clean.csv')
df

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Abstracts,special characters and punctuations removal,Numbers removal,Stopwords removal,Lower casing,Stemming,Lemmatization
0,\n Abstract not found\n ...,\n Abstract not found\n ...,\n Abstract not found\n ...,Abstract found,abstract not found,abstract not found,Abstract not found
1,\n describe a method for s...,\n describe a method for s...,\n describe a method for s...,describe method statistical modeling based max...,describe a method for statistical modeling bas...,describ a method for statist model base on max...,describe a method for statistical modeling bas...
2,\n Scaling conditional rand...,\n Scaling conditional rand...,\n Scaling conditional rand...,Scaling conditional random fields natural lang...,scaling conditional random fields for natural ...,scale condit random field for natur languag pr...,Scaling conditional random field for natural l...
3,\n The paper addresses the ...,\n The paper addresses the ...,\n The paper addresses the ...,The paper addresses issue cooperation linguist...,the paper addresses the issue of cooperation b...,the paper address the issu of cooper between l...,The paper address the issue of cooperation bet...
4,\n In most natural language...,\n In most natural language...,\n In most natural language...,"In natural language processing applications, D...",in most natural language processing applicatio...,"In most natur languag process applications, de...",In most natural language processing applicatio...
...,...,...,...,...,...,...,...
495,\n ABSTRACT: Web service us...,\n ABSTRACT Web service use...,\n ABSTRACT: Web service us...,ABSTRACT: Web service used Service Oriented Ar...,abstract: web service used in service oriented...,abstract: web servic use in servic orient arch...,ABSTRACT: Web service used in Service Oriented...
496,\n Much of the pre-existing...,\n Much of the preexisting ...,\n Much of the pre-existing...,Much pre-existing electronic data could harnes...,much of the pre-existing electronic data that ...,much of the pre-exist electron data that could...,Much of the pre-existing electronic data that ...
497,\n from sources with a hie...,\n from sources with a hie...,\n from sources with a hie...,sources hierarchical structure. The resulting ...,from sources with a hierarchical structure. th...,from sourc with a hierarch structure. the resu...,from source with a hierarchical structure. The...
498,\n . But learners get annoy...,\n But learners get annoye...,\n . But learners get annoy...,. But learners get annoyed language rules old ...,. but learners get annoyed with the language r...,. but learner get annoy with the languag rule ...,. But learner get annoyed with the language ru...


In [34]:
#compute the tf idf and tf-idf values for the starting word in the text
#computeTF
def TF(sentence):
  word = sentence.split(" ")
  value = len(set(word))
  return words.count(word[0])/value
  pass
#compute IDF
import math 
def IDF(sentence):
  words = sentence.lower().split(" ")
  res = words.count(words[0])
  return math.log(len(words)/res, 10)
  pass
df["word used for TF-IDF"]=df["Lemmatization"].apply(lambda x : x.split(" ")[0])
df["tf"]=df["Lemmatization"].apply(lambda x : round(TF(x),2))
df["idf"]=df["Lemmatization"].apply(lambda x : round(IDF(x),2))
df["tf-idf"]=df1["tf"]*df["idf"]
df

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
from textblob import TextBlob
nltk.download('punkt')
f=open("movies_list.csv","r")
df=pd.DataFrame(f)
df.columns=['ForEx5']
df['ForEx5']=df['ForEx5'].apply(lambda x: TextBlob(x).words)
dic1={}
for dff in df['ForEx5']:
  op=nltk.pos_tag(dff, tagset='universal')
  dic1.update(dict(op))
print(dic1)
'''
def count(POS):
  pos='{}'.format(POS)
  if v==pos:
    '{}_count'.format(POS)='{}_count'.format(POS)+1
  return c
for k,v in dic1.items():
  print(count('NOUN'))
'''
Noun_count=0
Verb_count=0
Adjective_count=0
Adverb_count=0
for key, value in dic1.items():
  if value=='NOUN':
    Noun_count+=1
  if value=='VERB':
    Verb_count+=1
  if value=='ADJ':
    Adjective_count+=1
  if value=='ADV':
    Adverb_count+=1
print('Total number of Nouns: {}\nTotal number of Verbs: {}\nTotal number of Adjectives: {}\nTotal number of Adverbs: {}\n'.format(Noun_count,Verb_count,Adjective_count,Adverb_count))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
{'movie_name': 'NOUN', '0': 'NUM', 'Dead': 'ADJ', 'Man': 'NOUN', '1': 'NUM', 'All': 'DET', 'Good': 'ADJ', 'Things': 'NOUN', '2': 'NUM', 'Brick': 'NOUN', '3': 'NUM', 'Breakfast': 'NOUN', 'at': 'ADP', 'Tiffany': 'NOUN', "'s": 'PRT', '4': 'NUM', 'Billy': 'NOUN', 'Elliot': 'NOUN', '5': 'NUM', 'Buried': 'NOUN', '6': 'NUM', 'The': 'DET', 'Crying': 'NOUN', 'Game': 'NOUN', '7': 'NUM', 'Case': 'NOUN', '39': 'NUM', '8': 'NUM', 'Crow': 'NOUN', '9': 'NUM', 'Downfall': 'NOUN', '10': 'NUM', 'Daddy': 'NOUN', 'Day': 'NOUN', 'Care': 'NOUN', '11,8MM': 'NUM', '12': 'NUM', 'Followi

In [24]:
#sentence length
def str_len(x):
  return len(x.split(" "))
df['sentence length']=[*map(str_len, df['Abstracts'])]

In [34]:
# Preposition Density
from collections import Counter
count=[]
df['Preposition']=in_counter=[x['IN'] for x in count]
df['Preposition_Density']=(df['Preposition']/df['sentence length'])*100
df
